# Logging and Monitoring Basics with tf.contrib.learn

https://www.tensorflow.org/versions/r0.11/tutorials/monitors/index.html

https://www.tensorflow.org/versions/r0.11/api_docs/python/contrib.learn.monitors.html

https://www.tensorflow.org/versions/r0.11/api_docs/python/contrib.learn.html#DNNClassifier

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

**Load the Iris CSV data to TensorFlow**

In [2]:
import os
import shutil

HOME_DIR = 'iris'
DATA_DIR = os.path.join(HOME_DIR, 'data')

IRIS_TRAINING = "iris_training.csv"
IRIS_TEST = "iris_test.csv"

TRAINING_FILE = os.path.join(DATA_DIR, IRIS_TRAINING)
TEST_FILE = os.path.join(DATA_DIR, IRIS_TEST)

from tensorflow.contrib.learn.python.learn.datasets.base import maybe_download
maybe_download(IRIS_TRAINING, DATA_DIR, 'http://download.tensorflow.org/data/iris_training.csv')
maybe_download(IRIS_TEST, DATA_DIR, 'http://download.tensorflow.org/data/iris_test.csv')

MODEL_DIR = os.path.join(HOME_DIR, 'model_log')

if os.path.isdir(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)

In [3]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TRAINING_FILE,
    target_dtype=np.int,
    features_dtype=np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=TEST_FILE,
    target_dtype=np.int,
    features_dtype=np.float32)

**Construct a Deep Neural Network Classifier**

In [4]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    n_classes=3,
    model_dir=MODEL_DIR,
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1)) #to ensure a sufficient number of checkpoints

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, '_is_chief': True, 'save_checkpoints_secs': 1, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, '_job_name': None, 'cluster_spec': None, 'tf_random_seed': None, 'num_ps_replicas': 0}


**Fit the DNNClassifier to the Iris Training Data**

In [5]:
#Customizing the Evaluation Metrics
# validation_metrics = {
#     "accuracy": tf.contrib.metrics.streaming_accuracy,
#     "precision": tf.contrib.metrics.streaming_precision,
#     "recall": tf.contrib.metrics.streaming_recall
# }

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    #metrics=validation_metrics,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=200)

classifier.fit(
    x=training_set.data,
    y=training_set.target,
    steps=2000,
    monitors=[validation_monitor])

INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(None), Dimension(4)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 1.0897, step = 1
INFO:tensorflow:Saving checkpoints for 1 into iris/model_log/model.ckpt.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Restored model from iris/model_log
INFO:tensorflow:Eval steps [0,inf) for training step 1.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 1 step: loss = 1.45386, accuracy = 0.266667
INFO:tensorflow:Validation (step 50): lo

Estimator(params={'enable_centered_bias': True, 'activation_fn': <function relu at 0x7f51c07bbc80>, 'weight_column_name': None, 'hidden_units': [10, 20, 10], 'feature_columns': [_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)], 'n_classes': 3, 'optimizer': 'Adagrad', 'dropout': None, 'gradient_clip_norm': None, 'num_ps_replicas': 0})

**Evaluate Model Accuracy**

In [6]:
accuracy_score = classifier.evaluate(x=test_set.data,y=test_set.target)['accuracy']
print('Accuracy: {0:f}'.format(accuracy_score))

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Restored model from iris/model_log
INFO:tensorflow:Eval steps [0,inf) for training step 1350.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 1350 step: loss = 0.0886846, accuracy = 0.966667


Accuracy: 0.966667


In [7]:
new_samples = np.array([[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=float)
y = classifier.predict(new_samples)
print('Predictions: {}'.format(str(y)))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Loading model from checkpoint: iris/model_log/model.ckpt-1350-?????-of-00001.


Predictions: [1 1]
